In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('diabetes.csv')

In [3]:
X = data.drop('Outcome', axis=1).values
y = data['Outcome'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)


train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

In [25]:
hidden_size1 = 128
hidden_size2 = 64
class DeepMLP(nn.Module):
    def __init__(self, input_size):
        super(DeepMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)  # Input to first hidden layer
        self.relu1 = nn.ReLU()  # Activation function for the first hidden layer
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)  # First hidden layer to second hidden layer
        self.relu2 = nn.ReLU()  # Activation function for the second hidden layer
        self.fc3 = nn.Linear(hidden_size2, 1)  # Second hidden layer to output layer

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return torch.sigmoid(x)

train_data = TensorDataset(X_train, y_train)
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

input_size = X_train.shape[1]
model = DeepMLP(input_size)

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 200
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/200], Loss: 0.7000212669372559
Epoch [2/200], Loss: 0.6463653445243835
Epoch [3/200], Loss: 0.6444919109344482
Epoch [4/200], Loss: 0.6578204035758972
Epoch [5/200], Loss: 0.5467489361763
Epoch [6/200], Loss: 0.5639954209327698
Epoch [7/200], Loss: 0.7988976836204529
Epoch [8/200], Loss: 0.542756199836731
Epoch [9/200], Loss: 0.5571168065071106
Epoch [10/200], Loss: 0.7009866833686829
Epoch [11/200], Loss: 0.7764363288879395
Epoch [12/200], Loss: 0.7050511240959167
Epoch [13/200], Loss: 0.5990083813667297
Epoch [14/200], Loss: 0.5183179378509521
Epoch [15/200], Loss: 0.4007430970668793
Epoch [16/200], Loss: 0.6280878782272339
Epoch [17/200], Loss: 0.5005929470062256
Epoch [18/200], Loss: 0.5375706553459167
Epoch [19/200], Loss: 0.7155391573905945
Epoch [20/200], Loss: 0.49015799164772034
Epoch [21/200], Loss: 0.8578360080718994
Epoch [22/200], Loss: 0.48283496499061584
Epoch [23/200], Loss: 0.3581743538379669
Epoch [24/200], Loss: 0.23918698728084564
Epoch [25/200], Loss: 0.25

In [29]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = (y_pred >= 0.5).float()
    accuracy = (y_pred == y_test.view(-1, 1)).float().mean()
    print(f'Accuracy on test set: {accuracy.item()}')

Accuracy on test set: 0.7402597665786743
